___
___
___
# RandomForestClassifier
___
___
___

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

df = pd.read_csv('./data_set/SBAnational-EDA.csv')
df = df.drop(columns=['Unnamed: 0'], axis=1).reset_index(drop=True)

df.columns

/home/utilisateur/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Index(['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'ApprovalDate',
       'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'MIS_Status',
       'GrAppv', 'SBA_Appv', 'NAICS2', 'DiffJobs'],
      dtype='object')

In [2]:
dd = df.groupby(['ApprovalFY', 'MIS_Status']).size().reset_index(name='count')

# Pivoter les résultats
dd = dd.pivot(index='ApprovalFY', columns='MIS_Status', values='count').reset_index()

# Remplacer les NaN par 0
dd.fillna(0, inplace=True)

# Créer une nouvelle colonne basée sur la comparaison 'CHGOFF' > 'P I F'
dd['ApprovalFY_bool'] = dd.apply(lambda row: 0 if row['CHGOFF'] > row['P I F'] else 1, axis=1)

dd = dd[['ApprovalFY_bool']]

df = pd.concat([df, dd], axis=1)

In [3]:
df = df[[
    'City',
    'State',
    'Zip',
    'Bank',
    'BankState',
    'NAICS',
    # 'NAICS2',
    'ApprovalDate',
    'ApprovalFY',
    # 'ApprovalFY_bool',
    'Term',
    'NoEmp',
    'NewExist',
    'CreateJob',
    'RetainedJob',
    'DiffJobs',
    'FranchiseCode',
    'UrbanRural',
    'RevLineCr',
    'LowDoc',
    'GrAppv',
    'SBA_Appv',
    'MIS_Status',
    ]]

In [4]:
df.head(1).transpose()

,0
City,EVANSVILLE
State,IN
Zip,47711
Bank,FIFTH THIRD BANK
BankState,IN
NAICS,451120
ApprovalDate,1997-02-28
ApprovalFY,1997-01-01
Term,84
NoEmp,4


In [5]:
colonnes_numeriques = df.select_dtypes(include=['number']).columns.tolist()

# Sélectionner les colonnes non numériques
colonnes_non_numeriques = df.drop(columns=colonnes_numeriques).columns.tolist()

print(colonnes_numeriques)
print(colonnes_non_numeriques)

['Zip', 'NAICS', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'DiffJobs', 'FranchiseCode', 'UrbanRural', 'GrAppv', 'SBA_Appv']
['City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY', 'RevLineCr', 'LowDoc', 'MIS_Status']


In [6]:
# Créer un nouveau DataFrame sans les lignes contenant des valeurs NaN
# df = df.dropna()

# Vérifier s'il y a des valeurs manquantes dans le DataFrame
missing_values = df.isnull().sum()
# Afficher les colonnes avec des valeurs manquantes
print("Colonnes avec des valeurs manquantes :\n", missing_values[missing_values > 0])

# Vérifier les types de données de chaque colonne
# data_types = df.dtypes
# Afficher les types de données de chaque colonne
# print("Types de données de chaque colonne :\n", data_types)

Colonnes avec des valeurs manquantes :
 State         6
Bank         25
BankState    25
dtype: int64


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier

grid_search_bool = False

numerical_features = make_column_selector(dtype_include=np.number)
categorical_features = make_column_selector(dtype_exclude=np.number)

# Définir le preprocessor
label_encoder = LabelEncoder()
preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(), StandardScaler()), numerical_features),
    (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop='if_binary')), categorical_features),
)

# Prétraiter les données
X = preprocessor.fit_transform(df.drop(columns='MIS_Status'))
y = label_encoder.fit_transform(df['MIS_Status'])

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)
# y_train = y_train.astype(int).tolist()
# y_test = y_test.astype(int).tolist()

if grid_search_bool == False:
    # LGBM
    d_train = lgb.Dataset(X_train, label=y_train)

    # https://lightgbm.readthedocs.io/en/latest/Parameters.html
    # best f1 macro : 0.9154
    # 

    # RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
    #                    criterion='gini', max_depth=None, max_features='sqrt',
    #                    max_leaf_nodes=None, max_samples=None,
    #                    min_impurity_decrease=0.0, min_samples_leaf=1,
    #                    min_samples_split=2, min_weight_fraction_leaf=0.0,
    #                    n_estimators=100, n_jobs=-1, oob_score=False,
    #                    random_state=0, verbose=0, warm_start=False)


    start=datetime.now()
    rf_classifier = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                criterion='gini', max_depth=None, max_features='sqrt',
                max_leaf_nodes=None, max_samples=None,
                min_impurity_decrease=0.0, min_samples_leaf=1,
                min_samples_split=2, min_weight_fraction_leaf=0.0,
                n_estimators=100, n_jobs=-1, oob_score=False,
                random_state=0, verbose=0, warm_start=False)
    rf_classifier.fit(X_train, y_train)
    stop=datetime.now()
    execution_time_rfc = stop-start

    #Prediction on test data
    # y_pred = rf_classifier.predict(X_test)

    #convert into binary values 0/1 for classification
    # y_pred_lgbm = [list(x).index(max(x)) for x in y_pred_lgbm]

    print(execution_time_rfc)

___
### PICKEL

In [ ]:
import pickle

filename = './model/modelRFC.pkl'

with open(filename, 'wb') as file:
    pickle.dump(rf_classifier, file)
